# Quantum Kernels Implementation

In this cell we construct a small synthetic data set inspired by the discrete logarithm problem (DLP), following the setting of Ref. [1].

We fix a prime modulus $p$ and work in the multiplicative group
$$
  \mathbb{Z}_p^{\ast} = \{1,2,\dots,p-1\}.
$$
First, we find a generator $g$ of $\mathbb{Z}_p^{\ast}$ and build a lookup table for the discrete logarithm
$$
  \log_g : \mathbb{Z}_p^{\ast} \to \{0,1,\dots,p-2\},
$$
so that for each $x$ we know the unique exponent $k$ with $g^k \equiv x \pmod{p}$.

We then choose a random ``secret shift'' $s \in \{0,\dots,p-2\}$ and define binary labels via
$$
  y(x) =
  \begin{cases}
    +1, & \text{if } (\log_g x - s) \bmod (p-1) < \dfrac{p-1}{2},\\[4pt]
    -1, & \text{otherwise.}
  \end{cases}
$$
Thus the exponent circle is cut into two contiguous halves, producing a balanced binary classification problem on $\mathbb{Z}_p^{\ast}$.

Each integer $x$ is then encoded as a bit string of length
$$
  n_{\text{bits}} = \bigl\lceil \log_2 p \bigr\rceil,
$$
which we interpret as input features suitable for an $n_{\text{bits}}$-qubit system.  Finally, we shuffle all $p-1$ points, split them into an $80{:}20$ train–test partition, and return
$(X_{\text{train}}, y_{\text{train}}, X_{\text{test}}, y_{\text{test}})$ together with a metadata dictionary containing $p$, $g$, $n_{\text{bits}}$, the secret shift $s$, and the integer indices used in the split.

In [1]:
import numpy as np
from math import ceil, log2
from typing import Tuple, Dict, Any

def is_generator(g: int, p: int) -> bool:
    """Check if g is a generator of Z_p^* (naive, fine for small p)."""
    residues = set(pow(g, k, p) for k in range(1, p))
    return len(residues) == p - 1

def find_generator(p: int) -> int:
    """Return a generator of Z_p^* (small p, brute force is fine)."""
    for g in range(2, p):
        if is_generator(g, p):
            return g
    raise ValueError("No generator found – p might not be prime?")

def discrete_log_table(p: int, g: int) -> Dict[int, int]:
    """
    Build a table mapping x -> k such that g^k ≡ x (mod p).
    """
    table: Dict[int, int] = {}
    value = 1  # g^0
    for k in range(p - 1):
        table[value] = k
        value = (value * g) % p
    return table

def int_to_bits(x: int, n_bits: int) -> np.ndarray:
    """Convert integer x to a binary vector of length n_bits (MSB first)."""
    return np.array([int(b) for b in format(x, f"0{n_bits}b")], dtype=float)

def generate_dlp_dataset(
    p: int = 97,              # <-- elegido para tener n_bits = 7
    train_fraction: float = 0.8,  # 80% train / 20% test
    seed: int = 0,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict[str, Any]]:
    """
    Generate a DLP-inspired dataset suitable for 7 qubits.

    Para el valor por defecto p=97:
      - n_bits  = ceil(log2(97)) = 7 qubits
      - n_total = p - 1 = 96 puntos (x = 1,...,96)
      - train_fraction = 0.8 -> n_train ≈ 77, n_test ≈ 19
    """
    rng = np.random.default_rng(seed)

    # 1) number-theory setup
    g = find_generator(p)                  # generator of Z_p^*
    log_tab = discrete_log_table(p, g)     # x -> k where g^k ≡ x (mod p)
    n_bits = ceil(log2(p))                 # number of qubits to represent 0..p-1

    # 2) secret shift s in exponent space
    s = rng.integers(0, p - 1)
    L = (p - 1) // 2                       # length of +1 interval

    def label_x(x: int) -> int:
        """
        Label based on exponent k = log_g(x):
          y = +1 if (k - s) mod (p-1) < L
              -1 otherwise
        """
        k = log_tab[x]
        return 1 if ((k - s) % (p - 1)) < L else -1

    # 3) sample all points and split into train/test
    all_x = np.arange(1, p)  # 1..p-1  (total = p-1 points)
    rng.shuffle(all_x)

    n_total = len(all_x)               # = p-1
    n_train = int(round(train_fraction * n_total))
    n_test  = n_total - n_train

    x_train_int = all_x[:n_train]
    x_test_int  = all_x[n_train:n_train + n_test]

    # 4) build features and labels
    X_train = np.vstack([int_to_bits(int(x), n_bits) for x in x_train_int])
    y_train = np.array([label_x(int(x)) for x in x_train_int])

    X_test = np.vstack([int_to_bits(int(x), n_bits) for x in x_test_int])
    y_test = np.array([label_x(int(x)) for x in x_test_int])

    meta: Dict[str, Any] = {
        "p": p,
        "g": g,
        "n_bits": n_bits,
        "s": int(s),
        "x_train_int": x_train_int,
        "x_test_int": x_test_int,
    }

    return X_train, y_train, X_test, y_test, meta

In [2]:
X_train, y_train, X_test, y_test, meta = generate_dlp_dataset()
print(meta)

{'p': 97, 'g': 5, 'n_bits': 7, 's': 81, 'x_train_int': array([96, 37, 21,  6, 24, 17, 75, 68, 53, 28, 40, 14, 92, 35, 12, 11, 81,
        9, 38, 10, 73, 63, 20, 91, 85, 84, 43, 88,  5, 26, 58, 95, 89, 45,
       56, 51, 69, 82, 16, 31,  3, 36, 61, 44, 18, 72, 29, 83, 19, 67,  4,
        2, 54, 25, 66, 22, 48,  1,  7, 65, 46, 23, 71, 27, 52, 50, 76, 90,
       93, 41, 62, 13, 33, 94, 47, 59, 15]), 'x_test_int': array([74, 39, 86, 32, 87, 49, 78, 77,  8, 64, 70, 79, 60, 55, 30, 42, 57,
       34, 80])}


# Quantum Part

We define a helper function, `build_feature_circuit_from_map`, to turn a parameterized Qiskit feature map (e.g., `ZFeatureMap` or `ZZFeatureMap`) into a concrete circuit for a specific data point $\mathbf{x}$.

For a given input vector $\mathbf{x} = (x_1,\dots,x_d)$, the function binds the circuit parameters to rotation angles:
$$
  \theta_i = \pi \cdot x_i.
$$
This scaling ensures that binary features map effectively ($0 \mapsto 0$ and $1 \mapsto \pi$). The function returns a bound `QuantumCircuit` representing the feature state $|\phi(x)\rangle$.

In [3]:
import numpy as np
from qiskit.circuit import QuantumCircuit

def build_feature_circuit_from_map(feature_map: QuantumCircuit,
                                   x_row: np.ndarray,
                                   angle_scale: float = np.pi) -> QuantumCircuit:
    """
    Given:
      - feature_map: a parametrized QuantumCircuit (e.g. ZZFeatureMap)
      - x_row: one data sample (shape [num_features])
    Return:
      - A new circuit where parameters have been assigned using x_row.
      
    We multiply each feature by angle_scale (e.g. 0/1 -> 0 or π).
    """
    # Get the parameters in a deterministic order (x[0], x[1], ...)
    params = sorted(feature_map.parameters, key=lambda p: p.name)
    assert len(params) == len(x_row), (
        f"Mismatch: feature_map expects {len(params)} features, "
        f"but got vector of length {len(x_row)}"
    )
    
    bind = {
        param: float(x_row[i]) * angle_scale
        for i, param in enumerate(params)
    }
    return feature_map.assign_parameters(bind)

This routine, `build_kernel_matrices_for_feature_map`, estimates kernel matrices using Qiskit’s `UnitaryOverlap` and `StatevectorSampler`.

For both training and test sets, it computes the kernel entries as the overlap fidelity between data points:
$$
  K(x_i,x_j) = |\langle\phi(x_i) | \phi(x_j)\rangle|^2 \approx \Pr(\text{measuring } 0\dots0).
$$
We construct the overlap circuit for each pair, estimate the probability of the all-zero outcome, and populate the train–train and test–train matrices to be used as precomputed kernels in an SVM.

In [4]:
from qiskit.circuit.library import UnitaryOverlap
from qiskit.primitives import StatevectorSampler

sampler = StatevectorSampler()  # reuse the same sampler

def build_kernel_matrices_for_feature_map(feature_map: QuantumCircuit,
                                          X_train: np.ndarray,
                                          X_test: np.ndarray,
                                          num_shots: int = 4096,
                                          angle_scale: float = np.pi):
    """
    For a given feature_map circuit and dataset (X_train, X_test),
    build:
      - kernel_matrix: train–train (n_train, n_train)
      - test_matrix:   test–train (n_test,  n_train)
    using UnitaryOverlap + StatevectorSampler.
    """
    n_train = X_train.shape[0]
    n_test  = X_test.shape[0]
    
    kernel_matrix = np.full((n_train, n_train), np.nan, dtype=float)
    test_matrix   = np.full((n_test,  n_train), np.nan, dtype=float)
    
    # Local helper using our generic builder
    def feature_circ(x_row):
        return build_feature_circuit_from_map(feature_map, x_row, angle_scale=angle_scale)
    
    def kernel_entry(x_i, x_j):
        """K(x_i, x_j) ~ Pr(measuring 0...0 in overlap circuit."""
        circ_i = feature_circ(x_i)
        circ_j = feature_circ(x_j)
        
        overlap_circ = UnitaryOverlap(circ_i, circ_j)
        overlap_circ.measure_all()
        
        result = sampler.run([overlap_circ], shots=num_shots).result()
        counts = result[0].data.meas.get_int_counts()
        return counts.get(0, 0) / num_shots
    
    # --- train–train ---
    for i in range(n_train):
        kernel_matrix[i, i] = 1.0  # ⟨φ(x)|φ(x)⟩ = 1
        for j in range(i + 1, n_train):
            val = kernel_entry(X_train[i], X_train[j])
            kernel_matrix[i, j] = val
            kernel_matrix[j, i] = val
    
    # --- test–train ---
    for i in range(n_test):
        for j in range(n_train):
            test_matrix[i, j] = kernel_entry(X_test[i], X_train[j])
    
    return kernel_matrix, test_matrix


We instantiate several standard Qiskit feature maps acting on $n_{\text{features}}$ qubits to compare different encoding strategies:

* **Angle Encoding:** `ZFeatureMap` (with 1 and 2 reps) using single-qubit $Z$-rotations.
* **Entangling:** `ZZFeatureMap` (with 1 and 2 reps) adding pairwise $ZZ$ interactions.
* **Expressive:** `PauliFeatureMap` using custom $XZ$ and $XYZ$ operator strings.

Each map defines a unique quantum kernel $K_{\text{fm}}(x,z) = |\langle\phi_{\text{fm}}(x) | \phi_{\text{fm}}(z)\rangle|^2$ which we will evaluate in the next step.

In [5]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, PauliFeatureMap

n_features = X_train.shape[1]  # should be 9

feature_maps = [
    ("ZFeatureMap_reps1",  ZFeatureMap(feature_dimension=n_features, reps=1)),
    ("ZFeatureMap_reps2",  ZFeatureMap(feature_dimension=n_features, reps=2)),
    ("ZZFeatureMap_reps1", ZZFeatureMap(feature_dimension=n_features,
                                        reps=1, entanglement="linear")),
    ("ZZFeatureMap_reps2", ZZFeatureMap(feature_dimension=n_features,
                                        reps=2, entanglement="linear")),
    ("Pauli_XZ_reps1",     PauliFeatureMap(feature_dimension=n_features,
                                           reps=1, paulis=['X', 'Z'])),
    ("Pauli_XYZ_reps1",    PauliFeatureMap(feature_dimension=n_features,
                                           reps=1, paulis=['X', 'Y', 'Z'])),
]

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
)
import numpy as np
import time

quantum_results = []   # will store a dict per feature map for later comparison

for name, fm in feature_maps:
    print(f"\n=== Feature map: {name} ===")
    
    # ---------------- Kernel construction ----------------
    t0 = time.time()
    K_train, K_test = build_kernel_matrices_for_feature_map(
        fm,
        X_train,
        X_test,
        num_shots=512,
        angle_scale=np.pi,   # 0/1 bits → 0 or π
    )
    kernel_time = time.time() - t0
    
    # Kernel statistics (off-diagonal)
    n_train = K_train.shape[0]
    diag_vals = np.diag(K_train)
    offdiag_vals = K_train[~np.eye(n_train, dtype=bool)]
    
    print(f"Kernel stats – diag mean={diag_vals.mean():.4f}, "
          f"diag std={diag_vals.std():.4f}")
    print(f"Kernel stats – off-diag mean={offdiag_vals.mean():.4f}, "
          f"off-diag std={offdiag_vals.std():.4f}, "
          f"min={offdiag_vals.min():.4f}, max={offdiag_vals.max():.4f}")
    print(f"Kernel build time: {kernel_time:.2f} s")
    
    # ---------------- Train SVM ----------------
    clf = SVC(kernel="precomputed", C=1.0)
    
    t1 = time.time()
    clf.fit(K_train, y_train)
    train_time = time.time() - t1
    
    # Train metrics
    y_train_pred = clf.predict(K_train)
    acc_train = accuracy_score(y_train, y_train_pred)
    
    # Test metrics
    y_pred = clf.predict(K_test)
    acc_test = accuracy_score(y_test, y_pred)
    
    # Detailed metrics from classification_report
    report = classification_report(y_test, y_pred, output_dict=True)
    macro_precision = report["macro avg"]["precision"]
    macro_recall    = report["macro avg"]["recall"]
    macro_f1        = report["macro avg"]["f1-score"]
    weighted_f1     = report["weighted avg"]["f1-score"]
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=[-1, 1])
    
    # Logging
    print(f"{name} – Train accuracy: {acc_train:.3f}")
    print(f"{name} – Test  accuracy: {acc_test:.3f}")
    print(f"{name} – Macro precision: {macro_precision:.3f}, "
          f"macro recall: {macro_recall:.3f}, macro F1: {macro_f1:.3f}")
    print(f"{name} – Weighted F1: {weighted_f1:.3f}")
    print("Confusion matrix (rows=true [-1, 1], cols=pred [-1, 1]):")
    print(cm)
    
    # If you still want the human-readable report:
    print("\nFull classification report:")
    print(classification_report(y_test, y_pred))
    
    # Store everything in a dict for later comparison / DataFrame
    quantum_results.append({
        "feature_map": name,
        "acc_train": acc_train,
        "acc_test": acc_test,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "weighted_f1": weighted_f1,
        "kernel_time": kernel_time,
        "train_time": train_time,
        "kernel_diag_mean": float(diag_vals.mean()),
        "kernel_diag_std": float(diag_vals.std()),
        "kernel_offdiag_mean": float(offdiag_vals.mean()),
        "kernel_offdiag_std": float(offdiag_vals.std()),
        "kernel_offdiag_min": float(offdiag_vals.min()),
        "kernel_offdiag_max": float(offdiag_vals.max()),
    })


=== Feature map: ZFeatureMap_reps1 ===
Kernel stats – diag mean=1.0000, diag std=0.0000
Kernel stats – off-diag mean=1.0000, off-diag std=0.0000, min=1.0000, max=1.0000
Kernel build time: 146.92 s
ZFeatureMap_reps1 – Train accuracy: 0.506
ZFeatureMap_reps1 – Test  accuracy: 0.474
ZFeatureMap_reps1 – Macro precision: 0.237, macro recall: 0.500, macro F1: 0.321
ZFeatureMap_reps1 – Weighted F1: 0.305
Confusion matrix (rows=true [-1, 1], cols=pred [-1, 1]):
[[ 9  0]
 [10  0]]

Full classification report:
              precision    recall  f1-score   support

          -1       0.47      1.00      0.64         9
           1       0.00      0.00      0.00        10

    accuracy                           0.47        19
   macro avg       0.24      0.50      0.32        19
weighted avg       0.22      0.47      0.30        19


=== Feature map: ZFeatureMap_reps2 ===


/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

Kernel stats – diag mean=1.0000, diag std=0.0000
Kernel stats – off-diag mean=1.0000, off-diag std=0.0000, min=1.0000, max=1.0000
Kernel build time: 188.32 s
ZFeatureMap_reps2 – Train accuracy: 0.506
ZFeatureMap_reps2 – Test  accuracy: 0.474
ZFeatureMap_reps2 – Macro precision: 0.237, macro recall: 0.500, macro F1: 0.321
ZFeatureMap_reps2 – Weighted F1: 0.305
Confusion matrix (rows=true [-1, 1], cols=pred [-1, 1]):
[[ 9  0]
 [10  0]]

Full classification report:
              precision    recall  f1-score   support

          -1       0.47      1.00      0.64         9
           1       0.00      0.00      0.00        10

    accuracy                           0.47        19
   macro avg       0.24      0.50      0.32        19
weighted avg       0.22      0.47      0.30        19


=== Feature map: ZZFeatureMap_reps1 ===


/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

Kernel stats – diag mean=1.0000, diag std=0.0000
Kernel stats – off-diag mean=0.6336, off-diag std=0.1684, min=0.2441, max=1.0000
Kernel build time: 207.43 s
ZZFeatureMap_reps1 – Train accuracy: 0.610
ZZFeatureMap_reps1 – Test  accuracy: 0.421
ZZFeatureMap_reps1 – Macro precision: 0.415, macro recall: 0.417, macro F1: 0.415
ZZFeatureMap_reps1 – Weighted F1: 0.418
Confusion matrix (rows=true [-1, 1], cols=pred [-1, 1]):
[[3 6]
 [5 5]]

Full classification report:
              precision    recall  f1-score   support

          -1       0.38      0.33      0.35         9
           1       0.45      0.50      0.48        10

    accuracy                           0.42        19
   macro avg       0.41      0.42      0.41        19
weighted avg       0.42      0.42      0.42        19


=== Feature map: ZZFeatureMap_reps2 ===
Kernel stats – diag mean=1.0000, diag std=0.0000
Kernel stats – off-diag mean=0.6112, off-diag std=0.1748, min=0.2090, max=1.0000
Kernel build time: 209.55 s
ZZFeatu

/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

Kernel stats – diag mean=1.0000, diag std=0.0000
Kernel stats – off-diag mean=1.0000, off-diag std=0.0000, min=1.0000, max=1.0000
Kernel build time: 186.00 s
Pauli_XYZ_reps1 – Train accuracy: 0.506
Pauli_XYZ_reps1 – Test  accuracy: 0.474
Pauli_XYZ_reps1 – Macro precision: 0.237, macro recall: 0.500, macro F1: 0.321
Pauli_XYZ_reps1 – Weighted F1: 0.305
Confusion matrix (rows=true [-1, 1], cols=pred [-1, 1]):
[[ 9  0]
 [10  0]]

Full classification report:
              precision    recall  f1-score   support

          -1       0.47      1.00      0.64         9
           1       0.00      0.00      0.00        10

    accuracy                           0.47        19
   macro avg       0.24      0.50      0.32        19
weighted avg       0.22      0.47      0.30        19



/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/juanp_baron11/Environments/qiskit-qml/qiskit-environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

# Classical Comparison

In this cell, we establish classical baselines using raw integer inputs rescaled to the interval $(0,1]$ via
$$
  \tilde{x} = \frac{x}{p}.
$$
We train two support vector machines on these 1D features:

* **Linear SVM:** assumes a linear decision boundary.
* **RBF SVM:** uses a Gaussian kernel to allow non-linear boundaries.

We report the test accuracy on held-out points to serve as a simple baseline using raw numeric input.

In [7]:
from sklearn.svm import SVC

# Raw integer features (scaled to [0,1])
x_train_raw = meta["x_train_int"].reshape(-1, 1) / meta["p"]
x_test_raw  = meta["x_test_int"].reshape(-1, 1) / meta["p"]

lin_raw = SVC(kernel="linear", C=1.0)
lin_raw.fit(x_train_raw, y_train)
acc_lin_raw = accuracy_score(y_test, lin_raw.predict(x_test_raw))
print(f"Linear SVM on raw x – accuracy: {acc_lin_raw:.3f}")

rbf_raw = SVC(kernel="rbf", gamma="scale", C=1.0)
rbf_raw.fit(x_train_raw, y_train)
acc_rbf_raw = accuracy_score(y_test, rbf_raw.predict(x_test_raw))
print(f"RBF SVM on raw x – accuracy: {acc_rbf_raw:.3f}")


Linear SVM on raw x – accuracy: 0.474
RBF SVM on raw x – accuracy: 0.474


We repeat the classical experiments using the bit-string encoding, where each example is a $d$-dimensional vector in $\{0,1\}^d$. We train:

* **Linear SVM** (`kernel="linear"`): searches for a hyperplane in the binary feature space.
* **RBF SVM** (`kernel="rbf"`): applies a Gaussian kernel to the bit vectors.

Comparing these results to the raw-integer baselines reveals whether the binary encoding exposes more useful structure to a classical classifier.

In [8]:
# Linear SVM on bit encodings
lin_bits = SVC(kernel="linear", C=1.0)
lin_bits.fit(X_train, y_train)
acc_lin_bits = accuracy_score(y_test, lin_bits.predict(X_test))
print(f"Linear SVM on bit features – accuracy: {acc_lin_bits:.3f}")

# RBF SVM on bit encodings
rbf_bits = SVC(kernel="rbf", gamma="scale", C=1.0)
rbf_bits.fit(X_train, y_train)
acc_rbf_bits = accuracy_score(y_test, rbf_bits.predict(X_test))
print(f"RBF SVM on bit features – accuracy: {acc_rbf_bits:.3f}")


Linear SVM on bit features – accuracy: 0.632
RBF SVM on bit features – accuracy: 0.474


Finally, we aggregate the test accuracies from all models—classical (raw and bits) and quantum. We collect the results into a list
$$
  \{(\texttt{name}, \mathrm{Acc}_{\text{test}})\}
$$
combining the four classical baselines with the quantum kernel results stored in `quantum_results`. This list is sorted in descending order and printed as a leaderboard to directly compare performance.

In [9]:
# Build a unified list: (name, metric) for classical + quantum

# Classical models (tuples)
all_results = [
    ("Linear SVM (raw x)",  acc_lin_raw),
    ("RBF SVM (raw x)",     acc_rbf_raw),
    ("Linear SVM (bits)",   acc_lin_bits),
    ("RBF SVM (bits)",      acc_rbf_bits),
]

# Add quantum models, using their test accuracy
all_results += [
    (qr["feature_map"], qr["acc_test"])
    for qr in quantum_results
]

print("\n=== Overall comparison by TEST accuracy (classical vs quantum) ===")
for name, acc in sorted(all_results, key=lambda x: x[1], reverse=True):
    print(f"{name:25s}  accuracy = {acc:.3f}")



=== Overall comparison by TEST accuracy (classical vs quantum) ===
Linear SVM (bits)          accuracy = 0.632
Linear SVM (raw x)         accuracy = 0.474
RBF SVM (raw x)            accuracy = 0.474
RBF SVM (bits)             accuracy = 0.474
ZFeatureMap_reps1          accuracy = 0.474
ZFeatureMap_reps2          accuracy = 0.474
Pauli_XZ_reps1             accuracy = 0.474
Pauli_XYZ_reps1            accuracy = 0.474
ZZFeatureMap_reps1         accuracy = 0.421
ZZFeatureMap_reps2         accuracy = 0.421


### Interpretation of Results

The leaderboard shows that the **Linear SVM on bit features** achieved the highest accuracy ($0.632$). Most other models, including the raw integer baselines and the majority of quantum kernels (Z and Pauli maps), clustered around an accuracy of $0.474$.

**Key Observations:**

1.  **Hardness of the Problem:** The Discrete Logarithm Problem generates data that appears pseudo-random. An accuracy of $\approx 0.47$ suggests those models are effectively guessing (near random chance for this dataset size).
2.  **Classical Advantage:** The slight edge of the Linear SVM (bits) suggests that the binary representation captures some marginal structure of the dividing line in the exponent circle that the raw integer scaling missed.
3.  **Quantum Performance:** The standard quantum kernels (`ZFeatureMap`, `ZZFeatureMap`, `PauliFeatureMap`) failed to outperform the classical baseline. This is expected; without a kernel specifically designed to exploit the algebraic structure of the discrete log (like Shor's period-finding structure), standard geometric embeddings perceive the cryptographically "hard" data as noise.

### References

[1] Y. Liu, S. Arunachalam, and K. Temme, "A rigorous and robust quantum speed-up in supervised machine learning," *Nature Physics*, vol. 17, no. 9, pp. 1013–1017, 2021. doi: 10.1038/s41567-021-01287-z.